In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pylab inline

In [ ]:
from loguru import logger

In [ ]:
import os
import sys
sys.path.append('..')

In [ ]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.evaluation import run_resample_experiment, _select_significant_entries, _compute_prec_rec_f1
from pals.common import *
from pals.feature_extraction import DataSource
from pals.PALS import PALS
from pals.ORA import ORA

# HAT Data Analysis

This notebook is used to generate resampled data and run the different methods for comparison in the manuscript. Results are analysed in `PALS_HAT_experiment_evaluation.ipynb`

### Load data

Assume token is stored in environmental variable *PIMP_API_TOKEN*

In [ ]:
token = get_pimp_API_token_from_env()

In [ ]:
analysis_id_plasma = 636
int_df_plasma, annotation_df_plasma, experimental_design_plasma = download_from_pimp(token, PIMP_HOST, analysis_id_plasma, 'kegg')

In [ ]:
analysis_id_csf = 635
int_df_csf, annotation_df_csf, experimental_design_csf = download_from_pimp(token, PIMP_HOST, analysis_id_csf, 'kegg')

### Create Data Sources

In [ ]:
database_name = DATABASE_PIMP_KEGG
ds_plasma = DataSource(int_df_plasma, annotation_df_plasma, experimental_design_plasma, database_name)
ds_csf = DataSource(int_df_csf, annotation_df_csf, experimental_design_csf, database_name)

In [ ]:
ds_plasma.get_measurements().shape

In [ ]:
ds_csf.get_measurements().shape

# Run PALS Experiments

In [ ]:
proportions = np.array([0.25, 0.50, 0.75])
num_iterations = 100
plage_weight = 5
hg_weight = 1
gsea_resamples = 1000

# pbar = True
# parallel = False

pbar = False
parallel = True

In [ ]:
results = {}

#### Disable debug logging

In [ ]:
set_log_level_info()

#### Run on Plasma samples

In [ ]:
data_source = ds_plasma
experiment_name = 'plasma'
case = 'Stage1'
control = 'Control'
significant_column = 'PiMP_KEGG Stage1/Control comb_p'

In [ ]:
res = run_resample_experiment(experiment_name, data_source, case, control, proportions, significant_column, num_iterations, plage_weight, hg_weight, gsea_resamples, 
                              pbar=pbar, parallel=parallel)
results[experiment_name] = res

#### Run on CSF samples

In [ ]:
data_source = ds_csf
experiment_name = 'csf'
case = 'Stage_2'
control = 'Control'
significant_column = 'PiMP_KEGG Stage_2/Control comb_p'

In [ ]:
res = run_resample_experiment(experiment_name, data_source, case, control, proportions, significant_column, num_iterations, plage_weight, hg_weight, gsea_resamples, 
                              pbar=pbar, parallel=parallel)
results[experiment_name] = res

#### Save Results

In [ ]:
save_obj(results, os.path.join('test_data', 'HAT', 'HAT_results.p'))